# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120432e+02     1.629466e+00
 * time: 0.0915229320526123
     1     1.020353e+01     9.280651e-01
 * time: 1.3865461349487305
     2    -1.163484e+01     1.072797e+00
 * time: 1.5205409526824951
     3    -3.403777e+01     7.536841e-01
 * time: 1.6778090000152588
     4    -4.730902e+01     5.833749e-01
 * time: 1.8325390815734863
     5    -5.678672e+01     2.421626e-01
 * time: 1.979464054107666
     6    -5.953226e+01     2.581652e-01
 * time: 2.0921881198883057
     7    -6.074551e+01     1.442324e-01
 * time: 2.19901704788208
     8    -6.130616e+01     6.457467e-02
 * time: 2.3142290115356445
     9    -6.158138e+01     3.965875e-02
 * time: 2.4219679832458496
    10    -6.179614e+01     2.936352e-02
 * time: 2.5381410121917725
    11    -6.192967e+01     2.145318e-02
 * time: 2.6459789276123047
    12    -6.202732e+01     1.783384e-02
 * time: 2.7638580799102783
    13    -6.207187e+01     1.462447e-02
 * time: 2.8782489299774

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671082
    AtomicLocal         -18.8557751
    AtomicNonlocal      14.8522687
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485400 
    Xc                  -19.3336831

    total               -62.261666459583
